Ищем строки с номинациями и удалить строки без номинаций (в ангийских контекстах)

In [1]:
import pandas as pd
import re

# Пути к файлам
corpus_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\15.02 corp\corp.xlsx"
names_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\nom.xlsx"
output_file = r"C:\Users\Sony\OneDrive\Рабочий стол\Переподготовка ВШЭ\проект\2\LOTR\15.02 corp\corp_nom.xlsx"

# Загружаем файлы
df_corpus = pd.read_excel(corpus_file)
df_names = pd.read_excel(names_file)

print(f"Корпус: {len(df_corpus)} строк")
print(f"Файл с номинациями: {len(df_names)} строк")

# Извлекаем английские номинации (первый столбец)
english_noms_raw = df_names.iloc[:, 0].dropna().tolist()

# Очищаем и нормализуем номинации
def clean_nom(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # Удаляем текст в скобках (часто варианты написания)
    text = re.sub(r'\(.*?\)', '', text)
    # Удаляем кавычки
    text = re.sub(r'["\'`]', '', text)
    # Убираем лишние пробелы
    text = ' '.join(text.split())
    return text.strip()

# Очищаем все номинации
english_noms = [clean_nom(nom) for nom in english_noms_raw if clean_nom(nom)]

# Удаляем дубликаты и пустые строки
english_noms = list(set([nom for nom in english_noms if nom]))
print(f"Уникальных английских номинаций: {len(english_noms)}")

# Покажем первые 10 для проверки
print("\nПримеры номинаций:")
for nom in sorted(english_noms)[:10]:
    print(f"  - {nom}")

# Функция для поиска номинаций в тексте
def find_english_noms(text, nom_list):
    if pd.isna(text):
        return []
    
    text_lower = str(text).lower()
    found = []
    
    for nom in nom_list:
        if not nom:
            continue
        nom_lower = nom.lower()
        
        # 1. Точное вхождение как отдельного слова
        pattern1 = r'\b' + re.escape(nom_lower) + r'\b'
        
        # 2. Вхождение с возможным окончанием (для притяжательных форм: Old Forest's)
        pattern2 = r'\b' + re.escape(nom_lower) + r'[\'’]?\w*\b'
        
        if re.search(pattern2, text_lower):
            found.append(nom)
    
    return found

# Применяем поиск ко всем строкам
df_corpus['found_noms'] = df_corpus['english'].apply(
    lambda x: find_english_noms(x, english_noms)
)

# Добавляем колонку-флаг: есть ли хотя бы одна номинация
df_corpus['has_nom'] = df_corpus['found_noms'].apply(lambda x: len(x) > 0)

# Считаем статистику
total_rows = len(df_corpus)
rows_with_nom = df_corpus['has_nom'].sum()
rows_without_nom = total_rows - rows_with_nom

print(f"Всего строк: {total_rows}")
print(f"Строк с номинациями: {rows_with_nom} ({rows_with_nom/total_rows*100:.1f}%)")
print(f"Строк без номинаций: {rows_without_nom} ({rows_without_nom/total_rows*100:.1f}%)")

# Несколько примеров найденных номинаций
print(f"\nПримеры строк с найденными номинациями:")
examples = df_corpus[df_corpus['has_nom']].head(10)
for idx, row in examples.iterrows():
    text = str(row['english'])[:100] + "..." if len(str(row['english'])) > 100 else str(row['english'])
    print(f"\nСтрока {idx}:")
    print(f"  Текст: {text}")
    print(f"  Найдено: {', '.join(row['found_noms'][:3])}")

# Фильтруем: оставляем только строки с номинациями
df_filtered = df_corpus[df_corpus['has_nom']].copy()

# Удаляем вспомогательные колонки перед сохранением
df_filtered = df_filtered.drop(columns=['found_noms', 'has_nom'])

# Сохраняем результат
print(f"\nСохраняем {len(df_filtered)} строк в файл...")
df_filtered.to_excel(output_file, index=False)




Корпус: 7251 строк
Файл с номинациями: 1596 строк
Уникальных английских номинаций: 1562

Примеры номинаций:
  - Accursed Years
  - Adelard
  - Adventure
  - Aglarond
  - Aiglos
  - Aldalómë
  - Aldor
  - Ambaróna
  - Amon Dîn
  - Amon Hen
Всего строк: 7251
Строк с номинациями: 4325 (59.6%)
Строк без номинаций: 2926 (40.4%)

Примеры строк с найденными номинациями:

Строка 0:
  Текст: A bell clanged; and from the Watchers there went up a high and dreadful wail.
  Найдено: Watcher

Строка 2:
  Текст: A boding of ill we thought it, my father and I, for no tidings had we heard of Boromir since he went...
  Найдено: Boromir, Watcher

Строка 4:
  Текст: A broad stairway climbed from the Deep up to the Rock and the rear-gate of the Hornburg.
  Найдено: Hornburg, Horn

Строка 6:
  Текст: A cliff frowned upon their right; to their left rose grey slopes, dim and shadowy in the late night.
  Найдено: Shadow

Строка 9:
  Текст: A deep boom rumbled through the City like thunder running in the clouds